In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

load_dotenv()

True

In [2]:
chain = (
    ChatPromptTemplate.from_template(
        """Given the user request below, classify it as either being about `Anime or Story` or `Other`.

Be more willing to classify `Anime or Story`
Do not respond with more than one word.

<request>
{request}
</request>

Classification:"""
    )
    | ChatOpenAI(model_name="gpt-3.5-turbo-0613")
    | StrOutputParser()
)

chain.invoke({"request": "I want a romantic and comedy show"})

'Anime or Story'

In [3]:
anime_chain = ChatPromptTemplate.from_template(
    """You are an anime recommender. \
    You recommend an anime based on the user's request.
    
    Request: {request}
    Answer:
    """
) | ChatOpenAI(model_name="gpt-3.5-turbo-0613")

general_chain = ChatPromptTemplate.from_template(
    """Respond to the following question:

    Request: {request}
    Answer:
    """
) | ChatOpenAI(model_name="gpt-3.5-turbo-0613")

In [4]:
def route(info):
    if "anime" in info["topic"].lower():
        return anime_chain
    else:
        return general_chain

In [5]:
full_chain = {"topic": chain, "request": lambda x: x["request"]} | RunnableLambda(
    route
) | StrOutputParser()

### Use cases

In [6]:
full_chain.invoke({"request": "recommend me an adventure anime"})

'As in the database told me, I recommend you watch "One Piece". It is an exciting adventure anime that follows the journey of Monkey D. Luffy and his crew as they search for the ultimate treasure, the One Piece. With a vast and immersive world, memorable characters, and thrilling battles, this anime will surely take you on a grand adventure.'

In [52]:
full_chain.invoke({"request": "tell me a fun facct about the sun"})

AIMessage(content='I luv u! Did you know that the sun is actually a giant ball of hot gas?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 42, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1359da09-402a-48f2-b5c2-3ca6881a68bd-0')